# ReAct Agent - LangGraph

Similar a arquitetura ReAct do create_agent do langchain, vamos construir step-by-step nosso loop ReAct com langgraph. 

(pensar->agir->observar->responder)

![Screenshot 2024-08-21 at 12.45.32 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab7453080e6802cd1703_agent-memory1.png)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Definição de tools

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add, multiply, divide]

## Iniciando o chat model

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4.1-mini")

llm_with_tools = llm.bind_tools(tools)

## State + Prompt + Node

In [4]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="Você é um assistente encarregado de realizar operações aritméticas em um conjunto de entradas.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

## definindo o grafo

In [5]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # Se a ultima mensagem (result) do assistant é uma tool call -> tools_condition roteia para tools
    # Se a ultima mensagem (result) do assistant não é uma tool call -> tools_condition roteia para END
    tools_condition,
)
# Retorna a resposta para o modelo (observe)
builder.add_edge("tools", "assistant")


## Memory 

Pra usar memoria, precisamos especificar um `thread_id`.

![state.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e0e9f526b41a4ed9e2d28b_agent-memory2.png)


In [6]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [7]:
react_graph_memory = builder.compile(checkpointer=memory)

In [8]:
# Specify a thread
config = {"configurable": {"thread_id": "1"}}

# Specify an input
messages = [HumanMessage(content="Faça a soma de 3 e 4.")]

# Run
messages = react_graph_memory.invoke({"messages": messages},config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Faça a soma de 3 e 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_LrvSiiwZnBXWnbEaJg8V0uU4)
 Call ID: call_LrvSiiwZnBXWnbEaJg8V0uU4
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

A soma de 3 e 4 é 7. Posso ajudar com mais alguma coisa?


In [9]:
messages = [HumanMessage(content="Multiplique o resultado por 2.")]
messages = react_graph_memory.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Faça a soma de 3 e 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_LrvSiiwZnBXWnbEaJg8V0uU4)
 Call ID: call_LrvSiiwZnBXWnbEaJg8V0uU4
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

A soma de 3 e 4 é 7. Posso ajudar com mais alguma coisa?
================================ Human Message =================================

Multiplique o resultado por 2.
================================== Ai Message ==================================
Tool Calls:
  multiply (call_4sH6j5R5n7zb4PWjMdyi9MOQ)
 Call ID: call_4sH6j5R5n7zb4PWjMdyi9MOQ
  Args:
    a: 7
    b: 2
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message =====